In [5]:
import numpy as np
import pandas as pd
import os
import cv2
from matplotlib import pyplot as plt
from PIL import Image
from skimage.draw import circle_perimeter
from skimage.morphology import skeletonize
from skimage import io
#from math import sqrt
import pickle

#import math
import fractal_dimension
import smoothing
import tortousity_measurement_f
from retina_final import Retina, Window, detect_vessel_border

In [8]:
def show_images(images):
    n: int = len(images)
    f = plt.figure()
    for i in range(n):
        # Debug, plot figure
        f.add_subplot(1, n, i + 1)
        plt.imshow(images[i],cmap='gray', vmin=0, vmax=255)

    plt.show(block=True)

In [9]:
def compute_zones(img_vessel,OD_center_x,OD_center_y,disc_width):
        zone_mask_B = np.zeros(img_vessel.shape)
        zone_mask_C = np.zeros(img_vessel.shape)
       
        zone_centre=(int(OD_center_x),int(OD_center_y))
        cv2.circle(zone_mask_B,zone_centre,radius=3*int(disc_width),color=(255,255,255),thickness=-1)
        cv2.circle(zone_mask_B,zone_centre,radius=2*int(disc_width),color=(0,0,0),thickness=-1)
        zone_mask_B = zone_mask_B/255
        binary_process_B = img_vessel*zone_mask_B 
        
        cv2.circle(zone_mask_C,zone_centre,radius=5*int(disc_width),color=(255,255,255),thickness=-1)
        cv2.circle(zone_mask_C,zone_centre,radius=2*int(disc_width),color=(0,0,0),thickness=-1)
        zone_mask_C = zone_mask_C/255            
        binary_process_C = img_vessel*zone_mask_C
        return binary_process_B,binary_process_C
  
    

In [10]:
def making_subject_wise_dict1(inf1, inf2, inf3, path_vessel ):
    
    inf1 = inf1.loc[inf1['patient name'] != '0']
    inf1 = inf1.reset_index(drop=True)

    inf2['patient name'] = inf2['patient name'].astype(str) + '.vol'
    information = pd.concat([inf1, inf2, inf3], axis = 0)
    #### find and drop duplicates     
    information = information.drop_duplicates(subset=['patient name'])
    # finding same subjects
    group = information.groupby('patient number')['patient name'].unique()
    list_group = list(group)
    
    id_path   = os.listdir(path_vessel)
    subjects_img = {}
    label = {}
    s = 0 # subject
    for subject in range(len(list_group)):
         # checking if the subject exist
         subject_exist = [y for y in id_path if list_group[subject][0].split('.')[0] in y]
         if subject_exist:
            name=[]
            for i in range(len(list_group[subject])):
          
               # finding the id of patients of the group in the id_path
               id_ = [y for y in id_path if list_group[subject][i].split('.')[0] in y][0]
               name.append(id_)

            subjects_img[s] = np.array(name) 
            ######################### labeling each group
            if ("Dr_Alamzadeh" in id_):            
                label[s] = 0
            elif ("hc" in id_):
                 label[s] = 0
            elif ("EYE" in id_):
                 label[s] = 0  
            elif ("ms" in id_):
                 label[s] = 1      
            else:
                 label[s] = 1
            s += 1
            
    return subjects_img, label



In [6]:
##read the path containing SLO images and binay vessel images
path_vessel='./final_vessel_result_/'##file containg binay vessel images
path_original='./all/all/'## file including SLO images

In [17]:
##loading disc and cup information
df_OD=pd.read_csv("OD_information.csv")
df_cup=pd.read_csv("cup_information.csv")
##loading SLO information in order to build subject-wise dictionary of SLO images
## you should change it according to your data
inf1=pd.read_csv("inf1.csv")
inf2=pd.read_csv("inf2.csv")
inf3=pd.read_csv("inf3.csv")

In [16]:
#making subject-wise data...
# this function should change according to your data
subjects_img,label=making_subject_wise_dict1(inf1,inf2,inf3,path_vessel)

In [12]:
count=0
images_name=[]
pixels_per_window=0
sampling_size = 0
r_2_threshold = 0
window_sizes = [512]
features={}
labels={}
name=os.listdir(path_vessel)[0].split("_")[0]
num_subject=0
feature_list=[]
number_subject=0
for filename in os.listdir(path_vessel):
   
    images_name.append(filename)
    ##binary vessel
    img = cv2.imread(os.path.join(path_vessel,filename)) 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img=cv2.resize(img,dsize=(512, 512), interpolation=cv2.INTER_CUBIC)
    
    
    img_original = cv2.imread(os.path.join(path_original,filename)) 
    img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2GRAY)
    img_original=cv2.resize(img_original,dsize=(512, 512), interpolation=cv2.INTER_CUBIC)
    
    img_vessel=np.where(img!=0,1,0)
    img_skeleton=np.uint8(skeletonize(img_vessel))
    segmentedImage = Retina(img_vessel, img_skeleton)
    window =Window(segmentedImage, window_sizes[-1], method="separated",min_pixels=pixels_per_window)
    FD_binary,VD_binary,Average_width,t1,t2, t4, td,vessels =tortousity_measurement_f.evaluate_window(
        window,min_pixels_per_vessel=10,sampling_size=6,r2_threshold=0.8)            
   ## print(FD_binary,VD_binary,Average_width,t2, t4, td)
    vessel_intensity= np.multiply(img_original,img_vessel).sum()/img_original.sum()
    #plt.imshow(img_skeleton)
    #for ii in range(len(vessels)):
        # plt.scatter(np.array(vessels[ii][1]), np.array(vessels[ii][0]), marker="x", color="red", s=1)
        
    
    ind=[ii for ii,x in enumerate(list(df_OD['image_name'])) if x==filename]
    if len(ind)==0:
        disc_width=-1
        OD_location_H=-1
        OD_location_V= -1
    if len(ind)!=0:
        #disc width
        disc_width=df_OD['OD_width'][ind[0]]
        #OD_center_x: col, OD_center_y:row
        OD_center_x=df_OD['OD_center_x'][ind[0]]
        OD_center_y=df_OD['OD_center_y'][ind[0]]
        ## disc location vertically ,disc location horizantally        
        OD_location_H=abs(np.subtract(int(img.shape[1]/2),OD_center_x))/img.shape[1]
        OD_location_V=abs(np.subtract(int(img.shape[0]/2),OD_center_y))/img.shape[0]
        ind1=[ii for ii,x in enumerate(list(df_cup['image_name'])) if x==filename]
     
        if len(ind1)!=0:
            ##cup_width
            cup_width=df_cup['cup_width'][ind1[0]]
            ##disc-cup width ratio
            OD_cup_width=disc_width/cup_width
        else:
            cup_width=-1
            OD_cup_width=-1
        ## zone B and zone C
        binary_process_B,binary_process_C=compute_zones(img_vessel,OD_center_x,OD_center_y,disc_width) 
        skelet_B=np.uint8(skeletonize(binary_process_B))
        skelet_C=np.uint8(skeletonize(binary_process_C))
        segmentedImage_B = Retina(binary_process_B, skelet_B)
        window_B=Window(segmentedImage_B, window_sizes[-1], method="separated",min_pixels=pixels_per_window)
        #show_images([binary_process_B*255,binary_process_C*255])
        FD_binary_B,VD_binary_B,Average_width_B,t1_B,t2_B, t4_B, td_B,vessels_B =tortousity_measurement_f.evaluate_window(
            window_B,min_pixels_per_vessel=10,sampling_size=6,r2_threshold=0.8) 
       ## print(FD_binary_B,VD_binary_B,Average_width_B,t1_B,t2_B, t4_B, td_B)
        segmentedImage_C = Retina(binary_process_C, skelet_C)
        window_C=Window(segmentedImage_C, window_sizes[-1], method="separated",min_pixels=pixels_per_window)
        #show_images([binary_process_B*255,binary_process_C*255])
        FD_binary_C,VD_binary_C,Average_width_C,t1_C,t2_C, t4_C, td_C,vessels_C =tortousity_measurement_f.evaluate_window(
            window_C,min_pixels_per_vessel=10,sampling_size=6,r2_threshold=0.8) 
        #print(FD_binary_C,VD_binary_C,Average_width_C,t1_C,t2_C, t4_C, td_C)
        
    
    for jj in range(len(subjects_img)):
        if filename in list(subjects_img[jj]):
            num_subject=jj
            break
    
    if count==0:
        feature_list.append([disc_width,cup_width,OD_cup_width, 
                                             Average_width,VD_binary,FD_binary,t2,t4,td,vessel_intensity,
                                             Average_width_B, Average_width_C,FD_binary_B,FD_binary_C,
                                             t1_B ,t1_C,t2_B,t2_C,t4_B,t4_C,td_B,td_C,VD_binary_B,VD_binary_C
                                             ])
        num_subject_old=num_subject
    else:
        if num_subject==num_subject_old:
           feature_list.append([disc_width,cup_width,OD_cup_width, 
                                             Average_width,VD_binary,FD_binary,t2,t4,td,vessel_intensity,
                                             Average_width_B, Average_width_C,FD_binary_B,FD_binary_C,
                                             t1_B ,t1_C,t2_B,t2_C,t4_B,t4_C,td_B,td_C,VD_binary_B,VD_binary_C
                                             ])
        else:
            features[number_subject]=np.array(feature_list)
            labels[number_subject]=label[num_subject]
            feature_list=[]
            num_subject_old=num_subject
            number_subject += 1
    
            
    
    if count==10:
        pass
    count +=1
    
#features_dic = pickle.dumps(features) 
    
    
    

In [13]:
#saving SLO features as pickle
with open('features_SLO.pkl', 'wb') as f:
    pickle.dump(features, f)

In [14]:
#saving labels corresponding to SLO features 
with open('labels_SLO.pkl', 'wb') as f:
    pickle.dump(labels, f)